<a href="https://colab.research.google.com/github/nuvita97/NLP-Machine-Translation/blob/main/NLP_Seq2Seq_%26_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!git clone https://github.com/nuvita97/NLP-Machine-Translation.git

Cloning into 'NLP-Machine-Translation'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [26]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import io
import string
import re
from sklearn.model_selection import train_test_split

In [27]:
path = 'NLP-Machine-Translation'
os.chdir(path)
os.listdir()

['README.md',
 '.git',
 'train.vi.txt',
 'NLP_Seq2Seq_&_Attention.ipynb',
 'train.en.txt']

In [28]:
en_filename = "train.en.txt"
vi_filename = "train.vi.txt"

raw_en_lines = open(en_filename, encoding='utf-8').read().strip().split("\n")
raw_vi_lines = open(vi_filename, encoding='utf-8').read().strip().split("\n")

print(len(raw_en_lines))
print(len(raw_vi_lines))

133317
133317


In [29]:
display(raw_en_lines[:3])
display(raw_vi_lines[:3])

['Rachel Pike : The science behind a climate headline',
 'In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .',
 'I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .']

['Khoa học đằng sau một tiêu đề về khí hậu',
 'Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cống hiến cho dự án này -- một chuyến bay mạo hiểm qua rừng già để tìm kiếm thông tin về một phân tử then chốt .',
 'Tôi muốn cho các bạn biết về sự to lớn của những nỗ lực khoa học đã góp phần làm nên các dòng tít bạn thường thấy trên báo .']

In [30]:
print("Punctuation : ", string.punctuation )
print("Digits : ", string.digits )

exclude = list(string.punctuation) + list(string.digits)
print("Exclude : ", exclude)

Punctuation :  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Digits :  0123456789
Exclude :  ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [31]:
def preprocess(sentence):
  sent = sentence.lower()
  sent = sent.strip()
  sent = re.sub("'", " ", sent)
  sent = re.sub("\s+", " ", sent)
  sent = ''.join([char for char in sent if char not in exclude])
  sent = "<start> " + sent + " <end>"
  return sent

preprocess("I go to school")

'<start> i go to school <end>'

In [32]:
en_lines = []
vi_lines = []
min_len, max_len = 10, 14

for eline, vline in zip(raw_en_lines, raw_vi_lines):
  eline = preprocess(eline)
  vline = preprocess(vline)
  if min_len < len(eline.split(" ")) < max_len and min_len < len(vline.split(" ")) < max_len:
    en_lines.append(eline)
    vi_lines.append(vline)
        
print(len(en_lines))
print(len(vi_lines))

6016
6016


In [33]:
class Language():
  def __init__(self, lines):
    self.lines = lines
    self.word2id = {}
    self.id2word = {}
    self.vocab = set()
    self.max_len = 0
    self.min_len = 0
    self.vocab_size = 0
    self.init_language_params()

  def init_language_params(self):
    for line in self.lines:
        self.vocab.update(line.split(" "))
    self.word2id['<pad>'] = 0
    for id, word in enumerate(self.vocab):
        self.word2id[word] = id + 1
    for word, id in self.word2id.items():
        self.id2word[id] = word
    self.max_len = max([len(line.split(" ")) for line in self.lines])
    self.min_len = min([len(line.split(" ")) for line in self.lines])
    self.vocab_size = len(self.vocab) + 1
          
  def sentence_to_vector(self, sent):
    return np.array([self.word2id[word] for word in sent.split(" ")])
          
  def vector_to_sentence(self, vector):
    return " ".join([self.id2word[id] for id in vector])

In [34]:
inp_lang = Language(en_lines)
tar_lang = Language(vi_lines)
print(inp_lang.max_len, inp_lang.min_len)
print(tar_lang.max_len, tar_lang.min_len)

inp_vector = [inp_lang.sentence_to_vector(line) for line in inp_lang.lines]
tar_vector = [tar_lang.sentence_to_vector(line) for line in tar_lang.lines]

inp_tensor = tf.keras.preprocessing.sequence.pad_sequences(inp_vector, inp_lang.max_len, padding='post')
tar_tensor = tf.keras.preprocessing.sequence.pad_sequences(tar_vector, tar_lang.max_len, padding='post')
print(inp_tensor.shape, tar_tensor.shape)

13 11
13 11
(6016, 13) (6016, 13)


In [35]:
x_train, x_val, y_train, y_val = train_test_split(inp_tensor, tar_tensor, test_size=0.1)

BATCH_SIZE = 32
BUFFER_SIZE = x_train.shape[0]
N_BATCH = BUFFER_SIZE//BATCH_SIZE
hidden_unit = 1024
embedding_size = 256
print(BUFFER_SIZE)


dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.batch(BATCH_SIZE)

5414


In [36]:
tmp_x, tmp_y = next(iter(dataset))
print(tmp_x.shape)
print(tmp_y.shape)

(32, 13)
(32, 13)


In [37]:
class Encode(tf.keras.Model):
  def __init__(self, embedding_size, vocab_size, hidden_units):
    super(Encode, self).__init__()
    self.Embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
    self.GRU = tf.keras.layers.GRU(
      hidden_units,
      return_sequences=True,
      return_state=True,
      recurrent_initializer='glorot_uniform')
    self.hidden_units = hidden_units
      
  def call(self, x, hidden_state):
    try:
      x = self.Embedding(x)
    except:
      print(x, print(inp_lang.vocab_size))          
    outputs, last_state = self.GRU(x, hidden_state)
    return outputs, last_state
  
  def init_hidden_state(self, batch_size):
    return tf.zeros([batch_size, self.hidden_units])

In [38]:
encoder = Encode(embedding_size, inp_lang.vocab_size, hidden_unit)
hidden_state = encoder.init_hidden_state(BATCH_SIZE)
tmp_outputs, last_state = encoder(tmp_x, hidden_state)
print(tmp_outputs.shape)
print(last_state.shape)

(32, 13, 1024)
(32, 1024)


In [39]:
class Attention(tf.keras.Model):
  def __init__(self, hidden_units):
    super(Attention, self).__init__()
    self.W_out_encode = tf.keras.layers.Dense(hidden_unit)
    self.W_state = tf.keras.layers.Dense(hidden_unit)
    self.V = tf.keras.layers.Dense(1)
      
  def call(self, encode_outs, pre_state):
    pre_state = tf.expand_dims(pre_state, axis=1)
    pre_state = self.W_state(pre_state)
    encode_outs = self.W_out_encode(encode_outs)
    score = self.V(tf.nn.tanh(pre_state + encode_outs))
    score = tf.nn.softmax(score, axis=1)
    context_vector = score*encode_outs
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector, score

In [40]:
attention = Attention(hidden_unit)
context_vector, attention_weight = attention(tmp_outputs, last_state)
print(context_vector.shape, attention_weight.shape)

(32, 1024) (32, 13, 1)


In [41]:
class Decode(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_units):
    super(Decode, self).__init__()
    self.hidden_units = hidden_units
    self.Embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
    self.Attention = Attention(hidden_units)
    self.GRU = tf.keras.layers.GRU(
        hidden_units,
        return_sequences=True,
        return_state=True,
        recurrent_initializer='glorot_uniform'
    )
    self.Fc = tf.keras.layers.Dense(vocab_size)
        
  def call(self, x, encode_outs, pre_state):
    x = tf.expand_dims(x, axis=1)
    try:
        x = self.Embedding(x)
    except:
        print(x, print(tar_lang.vocab_size))          
    context_vector, attention_weight = self.Attention(encode_outs, pre_state)
    context_vector = tf.expand_dims(context_vector, axis=1)
    gru_inp = tf.concat([x, context_vector], axis=-1)
    out_gru, state = self.GRU(gru_inp)
    out_gru = tf.reshape(out_gru, (-1, out_gru.shape[2]))
    return self.Fc(out_gru), state
  
  
decode = Decode(tar_lang.vocab_size, embedding_size, hidden_unit)
print(last_state.shape, tmp_outputs.shape, tmp_y[:, 0].shape)
decode_out, state = decode(tmp_y[:, 0], tmp_outputs, last_state)

(32, 1024) (32, 13, 1024) (32,)


In [42]:
def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [43]:
EPOCHS = 20
optimizer = tf.optimizers.Adam()
encoder = Encode(embedding_size, vocab_size=inp_lang.vocab_size, hidden_units=hidden_unit)
decoder = Decode(vocab_size=tar_lang.vocab_size, embedding_size=embedding_size, hidden_units=hidden_unit)

In [44]:
for epoch in range(EPOCHS):
  total_loss = 0
  for batch_id, (x, y) in enumerate(dataset.take(N_BATCH)):
    loss = 0
    with tf.GradientTape() as tape:
      first_state = encoder.init_hidden_state(batch_size=BATCH_SIZE)
      encode_outs, last_state = encoder(x, first_state)
      decode_state = last_state
      decode_input = [tar_lang.word2id["<start>"]]*BATCH_SIZE
      
      for i in range(1, y.shape[1]):
          decode_out, decode_state = decoder(decode_input, encode_outs, decode_state)
          loss += loss_function(y[:, i], decode_out)
          decode_input = y[:, i]
          
      train_vars = encoder.trainable_variables + decoder.trainable_variables
      grads = tape.gradient(loss, train_vars)
      optimizer.apply_gradients(zip(grads, train_vars))
    total_loss += loss
  print(total_loss.numpy())

10592.643
9103.253
8175.322
7314.9844
6507.0796
5784.626
5195.4585
4580.087
3982.1167
3466.7974
3082.3875
2742.4727
2393.151
2081.3403
1893.7026
1611.1245
1405.6572
1249.9858
1074.8694
927.9366


In [45]:
def translate(inputs):
  print(inp_lang.vector_to_sentence(inputs[0].numpy()))
  result = ''

  hidden = encoder.init_hidden_state(batch_size=1)
  enc_out, enc_hidden = encoder(inputs, hidden)
  dec_hidden = enc_hidden
  print(enc_out.shape, dec_hidden.shape)
  
  dec_input = [tar_lang.word2id['<start>']]
  for t in range(tar_lang.max_len):
      predictions, dec_hidden = decoder(dec_input, enc_out, dec_hidden)
      predicted_id = tf.argmax(predictions[0]).numpy()
      result += tar_lang.id2word[predicted_id] + ' '
      dec_input = [predicted_id]
  return result
  
for inp, tar in dataset.take(N_BATCH):
  print(translate(inp[5:6,:]))
  break

<start> but this has come as a great boon for them  <end>
(1, 13, 1024) (1, 1024)
nhưng điều này là một món quà tuyệt nhất cho chúng  <end> 
